In [257]:
import numpy as np
import pandas as pd
import seaborn as sns
import plotly.express as ex
import plotly.graph_objects as go
import plotly.figure_factory as ff

from matplotlib import pyplot as plt
from sklearn.pipeline import Pipeline
from sklearn.pipeline import FeatureUnion, FunctionTransformer
from sklearn.impute import SimpleImputer
from feature_engine.imputation import EndTailImputer
from sklearn.base import BaseEstimator, TransformerMixin

### load data into dataframe

In [258]:
house_data = pd.read_csv('../../data/external/train.csv')
house_data.drop('Id', axis=1, inplace=True)

In [259]:
house_data.head()

,MSSubClass,MSZoning,LotFrontage,LotArea,Street,Alley,LotShape,LandContour,Utilities,LotConfig,...,PoolArea,PoolQC,Fence,MiscFeature,MiscVal,MoSold,YrSold,SaleType,SaleCondition,SalePrice
0,60,RL,65.0,8450,Pave,NaN,Reg,Lvl,AllPub,Inside,...,0,NaN,NaN,NaN,0,2,2008,WD,Normal,208500
1,20,RL,80.0,9600,Pave,NaN,Reg,Lvl,AllPub,FR2,...,0,NaN,NaN,NaN,0,5,2007,WD,Normal,181500
2,60,RL,68.0,11250,Pave,NaN,IR1,Lvl,AllPub,Inside,...,0,NaN,NaN,NaN,0,9,2008,WD,Normal,223500
3,70,RL,60.0,9550,Pave,NaN,IR1,Lvl,AllPub,Corner,...,0,NaN,NaN,NaN,0,2,2006,WD,Abnorml,140000
4,60,RL,84.0,14260,Pave,NaN,IR1,Lvl,AllPub,FR2,...,0,NaN,NaN,NaN,0,12,2008,WD,Normal,250000


### levels of data

In [260]:
house_data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1460 entries, 0 to 1459
Data columns (total 80 columns):
 #   Column         Non-Null Count  Dtype  
---  ------         --------------  -----  
 0   MSSubClass     1460 non-null   int64  
 1   MSZoning       1460 non-null   object 
 2   LotFrontage    1201 non-null   float64
 3   LotArea        1460 non-null   int64  
 4   Street         1460 non-null   object 
 5   Alley          91 non-null     object 
 6   LotShape       1460 non-null   object 
 7   LandContour    1460 non-null   object 
 8   Utilities      1460 non-null   object 
 9   LotConfig      1460 non-null   object 
 10  LandSlope      1460 non-null   object 
 11  Neighborhood   1460 non-null   object 
 12  Condition1     1460 non-null   object 
 13  Condition2     1460 non-null   object 
 14  BldgType       1460 non-null   object 
 15  HouseStyle     1460 non-null   object 
 16  OverallQual    1460 non-null   int64  
 17  OverallCond    1460 non-null   int64  
 18  YearBuil

In [261]:
categorical_columns = house_data.select_dtypes(include=object).columns.tolist()
numerical_columns = house_data.select_dtypes(include=[np.int16, np.int32, np.int64,
                                                      np.float16, np.float32, np.float64]).columns.tolist()

In [262]:
np.array(categorical_columns)

array(['MSZoning', 'Street', 'Alley', 'LotShape', 'LandContour',
       'Utilities', 'LotConfig', 'LandSlope', 'Neighborhood',
       'Condition1', 'Condition2', 'BldgType', 'HouseStyle', 'RoofStyle',
       'RoofMatl', 'Exterior1st', 'Exterior2nd', 'MasVnrType',
       'ExterQual', 'ExterCond', 'Foundation', 'BsmtQual', 'BsmtCond',
       'BsmtExposure', 'BsmtFinType1', 'BsmtFinType2', 'Heating',
       'HeatingQC', 'CentralAir', 'Electrical', 'KitchenQual',
       'Functional', 'FireplaceQu', 'GarageType', 'GarageFinish',
       'GarageQual', 'GarageCond', 'PavedDrive', 'PoolQC', 'Fence',
       'MiscFeature', 'SaleType', 'SaleCondition'], dtype='<U13')

In [263]:
np.array(numerical_columns)

array(['MSSubClass', 'LotFrontage', 'LotArea', 'OverallQual',
       'OverallCond', 'YearBuilt', 'YearRemodAdd', 'MasVnrArea',
       'BsmtFinSF1', 'BsmtFinSF2', 'BsmtUnfSF', 'TotalBsmtSF', '1stFlrSF',
       '2ndFlrSF', 'LowQualFinSF', 'GrLivArea', 'BsmtFullBath',
       'BsmtHalfBath', 'FullBath', 'HalfBath', 'BedroomAbvGr',
       'KitchenAbvGr', 'TotRmsAbvGrd', 'Fireplaces', 'GarageYrBlt',
       'GarageCars', 'GarageArea', 'WoodDeckSF', 'OpenPorchSF',
       'EnclosedPorch', '3SsnPorch', 'ScreenPorch', 'PoolArea', 'MiscVal',
       'MoSold', 'YrSold', 'SalePrice'], dtype='<U13')

In [264]:
for col in categorical_columns:
    print('=' * 25, end='')
    print()
    print(col)
    print('=' * 25, end='')
    print()
    print(house_data[col].value_counts(normalize=True, dropna=False))
    print('-' * 25, end='')
    print()

MSZoning
RL         0.788356
RM         0.149315
FV         0.044521
RH         0.010959
C (all)    0.006849
Name: MSZoning, dtype: float64
-------------------------
Street
Pave    0.99589
Grvl    0.00411
Name: Street, dtype: float64
-------------------------
Alley
NaN     0.937671
Grvl    0.034247
Pave    0.028082
Name: Alley, dtype: float64
-------------------------
LotShape
Reg    0.633562
IR1    0.331507
IR2    0.028082
IR3    0.006849
Name: LotShape, dtype: float64
-------------------------
LandContour
Lvl    0.897945
Bnk    0.043151
HLS    0.034247
Low    0.024658
Name: LandContour, dtype: float64
-------------------------
Utilities
AllPub    0.999315
NoSeWa    0.000685
Name: Utilities, dtype: float64
-------------------------
LotConfig
Inside     0.720548
Corner     0.180137
CulDSac    0.064384
FR2        0.032192
FR3        0.002740
Name: LotConfig, dtype: float64
-------------------------
LandSlope
Gtl    0.946575
Mod    0.044521
Sev    0.008904
Name: LandSlope, dtype: float64

In [265]:
binary_columns = ['PavedDrive', 'CentralAir', 'Street']
nominal_columns = ['MSZoning', 'Alley', 'LandContour', 'Utilities',
                   'LotConfig', 'Neighborhood', 'Condition1',
                   'Condition2', 'BldgType', 'HouseStyle',
                   'RoofStyle', 'RoofMatl', 'Exterior1st', 'Exterior2nd',
                   'MasVnrType', 'Foundation', 'Heating', 'Electrical',
                   'Functional', 'GarageType', 'Fence', 'MiscFeature',
                   'SaleType']
ordinal_columns = ['MSSubClass', 'LotShape', 'LandSlope', 'OverallQual',
                   'OverallCond', 'ExterQual', 'ExterCond', 'BsmtQual',
                   'BsmtCond', 'BsmtExposure', 'BsmtFinType1', 'BsmtFinType2',
                   'HeatingQC', 'KitchenQual', 'FireplaceQu', 'GarageQual',
                   'GarageCond', 'PoolQC']

In [266]:
house_data[numerical_columns].isnull().mean().sort_values(ascending=False)

LotFrontage      0.177397
GarageYrBlt      0.055479
MasVnrArea       0.005479
OpenPorchSF      0.000000
KitchenAbvGr     0.000000
TotRmsAbvGrd     0.000000
Fireplaces       0.000000
GarageCars       0.000000
GarageArea       0.000000
WoodDeckSF       0.000000
MSSubClass       0.000000
BedroomAbvGr     0.000000
3SsnPorch        0.000000
ScreenPorch      0.000000
PoolArea         0.000000
MiscVal          0.000000
MoSold           0.000000
YrSold           0.000000
EnclosedPorch    0.000000
FullBath         0.000000
HalfBath         0.000000
BsmtFinSF2       0.000000
LotArea          0.000000
OverallQual      0.000000
OverallCond      0.000000
YearBuilt        0.000000
YearRemodAdd     0.000000
BsmtFinSF1       0.000000
BsmtUnfSF        0.000000
BsmtHalfBath     0.000000
TotalBsmtSF      0.000000
1stFlrSF         0.000000
2ndFlrSF         0.000000
LowQualFinSF     0.000000
GrLivArea        0.000000
BsmtFullBath     0.000000
SalePrice        0.000000
dtype: float64

In [267]:
house_data[categorical_columns].isnull().mean().sort_values(ascending=False)

PoolQC           0.995205
MiscFeature      0.963014
Alley            0.937671
Fence            0.807534
FireplaceQu      0.472603
GarageType       0.055479
GarageCond       0.055479
GarageQual       0.055479
GarageFinish     0.055479
BsmtFinType2     0.026027
BsmtExposure     0.026027
BsmtFinType1     0.025342
BsmtQual         0.025342
BsmtCond         0.025342
MasVnrType       0.005479
Electrical       0.000685
Functional       0.000000
KitchenQual      0.000000
CentralAir       0.000000
HeatingQC        0.000000
Heating          0.000000
PavedDrive       0.000000
SaleType         0.000000
MSZoning         0.000000
Street           0.000000
Condition2       0.000000
LotShape         0.000000
LandContour      0.000000
Utilities        0.000000
LotConfig        0.000000
LandSlope        0.000000
Neighborhood     0.000000
Condition1       0.000000
BldgType         0.000000
Foundation       0.000000
HouseStyle       0.000000
RoofStyle        0.000000
RoofMatl         0.000000
Exterior1st 

In [268]:
missing_cat_columns = ['Alley', 'MasVnrType', 'BsmtQual', 'BsmtCond',
                       'BsmtExposure', 'BsmtFinType1', 'BsmtFinType2',
                       'FireplaceQu', 'GarageType', 'GarageFinish',
                       'GarageQual', 'GarageCond', 'PoolQC',
                       'Fence', 'MiscFeature']
missing_num_columns = ['LotFrontage', 'GarageYrBlt']

In [269]:
house_data[missing_cat_columns]

,Alley,MasVnrType,BsmtQual,BsmtCond,BsmtExposure,BsmtFinType1,BsmtFinType2,FireplaceQu,GarageType,GarageFinish,GarageQual,GarageCond,PoolQC,Fence,MiscFeature
0,NaN,BrkFace,Gd,TA,No,GLQ,Unf,NaN,Attchd,RFn,TA,TA,NaN,NaN,NaN
1,NaN,None,Gd,TA,Gd,ALQ,Unf,TA,Attchd,RFn,TA,TA,NaN,NaN,NaN
2,NaN,BrkFace,Gd,TA,Mn,GLQ,Unf,TA,Attchd,RFn,TA,TA,NaN,NaN,NaN
3,NaN,None,TA,Gd,No,ALQ,Unf,Gd,Detchd,Unf,TA,TA,NaN,NaN,NaN
4,NaN,BrkFace,Gd,TA,Av,GLQ,Unf,TA,Attchd,RFn,TA,TA,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1455,NaN,None,Gd,TA,No,Unf,Unf,TA,Attchd,RFn,TA,TA,NaN,NaN,NaN
1456,NaN,Stone,Gd,TA,No,ALQ,Rec,TA,Attchd,Unf,TA,TA,NaN,MnPrv,NaN
1457,NaN,None,TA,Gd,No,GLQ,Unf,Gd,Attchd,RFn,TA,TA,NaN,GdPrv,Shed
1458,NaN,None,TA,TA,Mn,GLQ,Rec,NaN,Attchd,Unf,TA,TA,NaN,NaN,NaN


In categorical missing values there are two types. one is columns has a defined meaning of NA and other type is didn't have a defined meaning to NA's. we can classify first type as missing not at random and second as missing at random.

In [270]:
arbitrary_impute = SimpleImputer(strategy='constant', fill_value='None')
most_freq_impute = SimpleImputer(strategy='most_frequent')

In [271]:
arbitrary_impute.fit_transform(house_data[missing_cat_columns])

array([['None', 'BrkFace', 'Gd', ..., 'None', 'None', 'None'],
       ['None', 'None', 'Gd', ..., 'None', 'None', 'None'],
       ['None', 'BrkFace', 'Gd', ..., 'None', 'None', 'None'],
       ...,
       ['None', 'None', 'TA', ..., 'None', 'GdPrv', 'Shed'],
       ['None', 'None', 'TA', ..., 'None', 'None', 'None'],
       ['None', 'None', 'TA', ..., 'None', 'None', 'None']], dtype=object)

In [272]:
most_freq_impute.fit_transform(house_data[['Electrical']])

array([['SBrkr'],
       ['SBrkr'],
       ['SBrkr'],
       ...,
       ['SBrkr'],
       ['FuseA'],
       ['SBrkr']], dtype=object)

numerical feature imputation can be using mean, median or end of tail values

In [273]:
def grouped_hist(df, x, heu, kde=False, separate_cols=False):
    data_frame = df.copy()[[x, heu]].dropna()

    if kde:
        hist_data = [data_frame[data_frame[heu] == u_val][x] for u_val in data_frame[heu].unique()]
        g = ff.create_distplot(hist_data=hist_data,
                               group_labels=list(data_frame[heu].unique()),
                               show_hist=False)

        return g.show()

    if separate_cols:
        g = ex.histogram(data_frame=data_frame,
                         x=x,
                         facet_col=heu)

        return g.show()

    g = ex.histogram(data_frame=data_frame,
                     x=x,
                     color=heu)

    return g.show()

In [274]:
grouped_hist(house_data, 'LotFrontage', 'MSZoning', kde=True)

In [275]:
house_data.groupby(by='MSZoning')['LotFrontage'].aggregate(['mean', 'median'])

,mean,median
MSZoning,,
C (all),69.700000,60.0
FV,59.491228,65.0
RH,58.923077,60.0
RL,74.680043,72.0
RM,52.366834,51.0


In [276]:
grouped_hist(house_data, 'MasVnrArea', 'MasVnrType')

In [277]:
house_data.groupby(by='MasVnrType')['MasVnrArea'].aggregate(['mean', 'median'])

,mean,median
MasVnrType,,
BrkCmn,247.666667,192.0
BrkFace,259.008989,202.0
None,1.094907,0.0
Stone,239.304688,206.5


In [278]:
class InfraredMedianImputer(BaseEstimator, TransformerMixin):
    def __init__(self, anchor_map):
        self.impute_with = None
        self.anchor_column_map = anchor_map

    @staticmethod
    def get_anchor_median_map(df, x, anchor):
        try:
            return df.groupby(anchor)[x].median()
        except:
            return {}

    def fit(self, x, y=None):
        self.impute_with = {}

        if isinstance(x, pd.DataFrame):
            if isinstance(self.anchor_column_map, dict):
                for c, a in self.anchor_column_map.items():
                    self.impute_with[c] = x.groupby(by=a)[c].median().to_dict()
            elif isinstance(self.anchor_column_map, str):
                for c in x.columns:
                    self.impute_with[c] = x.groupby(by=self.anchor_column_map)[c].median().to_dict()
            else:
                AttributeError('anchor_map need to be either string or dictionary.')
        else:
            raise AttributeError('x need to be a DataFrame.')

        return self

    def transform(self, x, y=None):
        df = x.copy()

        for c, m in self.impute_with.items():
            for k, v in m.items():
                df[c] = np.where((df[c].isna()) & (df[self.anchor_column_map[c]] == k), v, df[c])

        return df

In [279]:
median_impute = InfraredMedianImputer({'LotFrontage': 'MSZoning',
                                       'MasVnrArea': 'MasVnrType'})
end_tail_impute = EndTailImputer(tail='right')

In [280]:
median_impute.fit_transform(house_data[['LotFrontage', 'MSZoning', 'MasVnrArea', 'MasVnrType']])

,LotFrontage,MSZoning,MasVnrArea,MasVnrType
0,65.0,RL,196.0,BrkFace
1,80.0,RL,0.0,None
2,68.0,RL,162.0,BrkFace
3,60.0,RL,0.0,None
4,84.0,RL,350.0,BrkFace
...,...,...,...,...
1455,62.0,RL,0.0,None
1456,85.0,RL,119.0,Stone
1457,66.0,RL,0.0,None
1458,68.0,RL,0.0,None


In [281]:
end_tail_impute.fit_transform(house_data[['MasVnrArea']])

,MasVnrArea
0,196.0
1,0.0
2,162.0
3,0.0
4,350.0
...,...
1455,0.0
1456,119.0
1457,0.0
1458,0.0


### feature improvement (imputation) pipelines

In [282]:
imputing_cols = [*missing_cat_columns,
                 'Electrical', 'LotFrontage',
                 'MSZoning', 'MasVnrArea',
                 'MasVnrType', 'GarageYrBlt']

categorical_impute_pipeline_1 = Pipeline([
    ('categorical_features', FunctionTransformer(lambda df: df[missing_cat_columns])),
    ('mnar_impute', SimpleImputer(strategy='constant', fill_value='None'))
])

categorical_impute_pipeline_2 = Pipeline([
    ('categorical_features', FunctionTransformer(lambda df: df[['Electrical']])),
    ('mar_impute', SimpleImputer(strategy='most_frequent'))
])

numerical_impute_pipeline_1 = Pipeline([
    ('numerical_features', FunctionTransformer(lambda df: df[['LotFrontage', 'MSZoning',
                                                              'MasVnrArea', 'MasVnrType']])),
    ('median_impute', InfraredMedianImputer({'LotFrontage': 'MSZoning',
                                             'MasVnrArea': 'MasVnrType'}))
])

numerical_impute_pipeline_2 = Pipeline([
    ('numerical_features', FunctionTransformer(lambda df: df[['GarageYrBlt']])),
    ('end_tail_impute', EndTailImputer(tail='right'))
])

cat_full_pipes = Pipeline([('union', FeatureUnion([('impute_1', categorical_impute_pipeline_1),
                                                   ('impute_2', categorical_impute_pipeline_2)])),
                           ('to_df', FunctionTransformer(lambda array: pd.DataFrame(array,
                                                                                    columns=[
                                                                                        *missing_cat_columns,
                                                                                        'Electrical']))),
                           ('final_impute', FunctionTransformer(lambda df: df.fillna(method='ffill')))])

num_full_pipes = Pipeline([('union', FeatureUnion([('impute_1', numerical_impute_pipeline_1),
                                                   ('impute_2', numerical_impute_pipeline_2)])),
                           ('to_df', FunctionTransformer(lambda array: pd.DataFrame(array,
                                                                                    columns=['LotFrontage', 'MSZoning',
                                                                                             'MasVnrArea', 'MasVnrType',
                                                                                             'GarageYrBlt']))),
                           ('final_impute', FunctionTransformer(lambda df: df.fillna(method='ffill')))])

full_impute_pipeline = Pipeline([('concat_pipes', FeatureUnion([('cat_imputer', cat_full_pipes),
                                                                ('num_imputer', num_full_pipes)]))])

In [283]:
house_data[imputing_cols] = full_impute_pipeline.fit_transform(house_data)